In [1]:
import pandas as pd
import hillmaker as hm
from pandas import DataFrame
from pandas import Timestamp
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import Minute

In [2]:
import itertools

In [3]:
from hillmaker import bydatetime
from hillmaker import hmlib

In [4]:
import numpy as np

In [ ]:
help(bydatetime.make_bydatetime)

In [2]:
catfield = 'PatType'

In [3]:
list(catfield)

['P', 'a', 't', 'T', 'y', 'p', 'e']

In [5]:
catfield = [catfield]

In [6]:
catfield

['PatType']

In [9]:
totfield = ['total']*10

In [5]:
CONST_FAKE_CATFIELDNAME = 'FakeCatForTotals'

In [5]:
catfield = [CONST_FAKE_CATFIELDNAME]
catfield

['FakeCatForTotals']

In [6]:
file_stopdata = '../data/ShortStay2.csv'

scenario = 'iss17_test1'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start_analysis_dt = '1/1/1996'
end_analysis_dt = '3/30/1996 23:45'

# Optional inputs
tot_fld_name = 'SSU'
bin_size_minutes = 60

In [7]:
stops_df = pd.read_csv(file_stopdata, parse_dates=[in_fld_name, out_fld_name])

In [9]:
stops_df.index.name

In [15]:
# Example from pandas doc for trying to figure out how index names work for multiindex.
iterables = [['bar', 'baz', 'foo', 'qux'], ['one', 'two']]
test_multlidx = pd.MultiIndex.from_product(iterables, names=['first', 'second'])
print(test_multlidx)
print('name:= ',test_multlidx.name)
print('names:= ',test_multlidx.names)

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])
name:=  None
names:=  ['first', 'second']


In [7]:
len(stops_df)

59877

In [15]:
stops_df.tail(4)

,PatID,InRoomTS,OutRoomTS,PatType,Severity
59873,59874,1996-09-30 20:23:00,1996-09-30 21:30:00,IVT,2
59874,59875,1996-09-30 21:00:00,1996-09-30 22:45:00,CAT,1
59875,59876,1996-09-30 21:57:00,1996-09-30 22:40:00,IVT,2
59876,59877,1996-09-30 22:45:00,1996-09-30 23:35:00,CAT,1


In [8]:
rng_bydt = pd.date_range(start_analysis_dt, end_analysis_dt, freq=Minute(bin_size_minutes))

In [9]:
len(rng_bydt)

2160

In [10]:
pd.Series(rng_bydt).head()

0   1996-01-01 00:00:00
1   1996-01-01 01:00:00
2   1996-01-01 02:00:00
3   1996-01-01 03:00:00
4   1996-01-01 04:00:00
dtype: datetime64[ns]

In [9]:
catfield = ['PatType','Severity']

In [12]:
catfield[1]

'Severity'

In [13]:
categories = []
cat_to_exclude = [[],[]]
if cat_to_exclude is not None:
        for i in range(len(catfield)):
            categories.append(tuple([c for c in stops_df[catfield[i]].unique() if c not in cat_to_exclude[i]]))
else:
        for i in range(len(catfield)):
            categories.append(tuple([c for c in stops_df[catfield[i]].unique()]))


In [14]:
categories

[('IVT', 'MYE', 'CAT', 'ART', 'OTH'), (1, 2)]

In [15]:
numcombos = 1
for c in categories:
    numcombos = numcombos * len(c)
print(numcombos)

10


In [16]:
# Only include stops records that have their categories included
for i in range(len(catfield)):
    stops_df = stops_df[stops_df[catfield[i]].isin(categories[i])]

In [17]:
# Create a list of column names for the by datetime table and then an empty data frame based on these columns.
# The column names for the category fields are now the actual field names
columns = []
measures = ['datetime', 'arrivals', 'departures', 'occupancy']
for field in catfield:
    columns.append(field)

columns.extend(measures)

bydt_df = DataFrame(columns=columns)


In [18]:
print(columns)

['PatType', 'Severity', 'datetime', 'arrivals', 'departures', 'occupancy']


In [19]:
bydt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
PatType       0 non-null object
Severity      0 non-null object
datetime      0 non-null object
arrivals      0 non-null object
departures    0 non-null object
occupancy     0 non-null object
dtypes: object(6)
memory usage: 0.0+ bytes


In [20]:
itertools.product(('IVT', 'MYE', 'CAT', 'ART', 'OTH'),(1,2))

In [21]:
for p in itertools.product(['IVT', 'MYE', 'CAT', 'ART', 'OTH'],[1,2]):
    print(p)

('IVT', 1)
('IVT', 2)
('MYE', 1)
('MYE', 2)
('CAT', 1)
('CAT', 2)
('ART', 1)
('ART', 2)
('OTH', 1)
('OTH', 2)


In [22]:
for p in itertools.product(*categories):
    print(p)
    print(*p)

('IVT', 1)
IVT 1
('IVT', 2)
IVT 2
('MYE', 1)
MYE 1
('MYE', 2)
MYE 2
('CAT', 1)
CAT 1
('CAT', 2)
CAT 2
('ART', 1)
ART 1
('ART', 2)
ART 2
('OTH', 1)
OTH 1
('OTH', 2)
OTH 2


In [23]:
print(*categories)

('IVT', 'MYE', 'CAT', 'ART', 'OTH') (1, 2)


In [24]:
for p in itertools.product(('IVT', 'MYE', 'CAT', 'ART', 'OTH'),(1,2)):
    print([*p],p[0],p[1])

['IVT', 1] IVT 1
['IVT', 2] IVT 2
['MYE', 1] MYE 1
['MYE', 2] MYE 2
['CAT', 1] CAT 1
['CAT', 2] CAT 2
['ART', 1] ART 1
['ART', 2] ART 2
['OTH', 1] OTH 1
['OTH', 2] OTH 2


In [ ]:
df1 = pd.DataFrame({'B': ['B0', 'B1', 'B2', 'B3'],
   ...:                     'D': ['D0', 'D1', 'D2', 'D3']},
   ...:                     index=[0, 1, 2, 3])

In [ ]:
df1 = pd.DataFrame({'B': ['B0', 'B1', 'B2', 'B3'],
   ...:                     'D': ['D0', 'D1', 'D2', 'D3']},
   ...:                     index=[0, 1, 2, 3])

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
   ...:                     'C': ['C0', 'C1', 'C2', 'C3']},               
   ...:                     index=[0, 1, 2, 3])

In [ ]:
df2

In [ ]:
pd.concat([df1,df2],axis=1)

In [ ]:
print(catfield)

In [ ]:
rng_bydt = pd.Series(pd.date_range(start_analysis_dt, end_analysis_dt, freq=Minute(bin_size_minutes)))

In [ ]:
itertools.product(*categories)

In [ ]:
i=2
print(i)
i+=1
print(i)

In [30]:
len_bydt = len(rng_bydt)
all_cat_df = []
for p in itertools.product(*categories):
        i=0
        cat_df = DataFrame()
        j=0
        for c in [*p]:
            print(c)
            bydt_catdata = {catfield[j]: [c] * len_bydt}
            cat_df_cat = DataFrame(bydt_catdata, columns=[catfield[j]])
            j+=1
            #cat_df_cat.info()
            cat_df = pd.concat([cat_df, cat_df_cat],axis=1)
        all_cat_df.append(cat_df)
        i+=1

IVT
1
IVT
2
MYE
1
MYE
2
CAT
1
CAT
2
ART
1
ART
2
OTH
1
OTH
2


In [ ]:
all_cat_df[1]

In [ ]:
cat_df.info()

In [ ]:
bydt_data = {'datetime': rng_bydt, 'arrivals': [0.0] * len_bydt,
                     'departures': [0.0] * len_bydt, 'occupancy': [0.0] * len_bydt}

In [ ]:
bydt_data['datetime'][10:20]

In [ ]:
bydt_data_df = DataFrame(bydt_data, columns=['datetime', 'arrivals', 'departures', 'occupancy'])

In [ ]:
bydt_data_df.info()

In [ ]:
bydt_data_df.head()

In [ ]:
bydt_df = DataFrame()
for cat_df in all_cat_df:
   
        #bydt_df_cat = DataFrame(bydt_data, columns=['category', 'datetime', 'arrivals', 'departures', 'occupancy'])
        bydt_df_cat = pd.concat([cat_df, bydt_data_df],axis=1)
        bydt_df = pd.concat([bydt_df, bydt_df_cat])

In [ ]:
len_bydt

In [ ]:
bydt_df.info()

In [26]:
bydt_df.head()

,PatType,Severity,datetime,arrivals,departures,occupancy


In [28]:
rng_bydt = pd.Series(pd.date_range(start_analysis_dt, end_analysis_dt, freq=Minute(bin_size_minutes)))
len_bydt = len(rng_bydt)

In [31]:
bydt_df = DataFrame()
bydt_data = {'datetime': rng_bydt, 'arrivals': [0.0] * len_bydt,
                         'departures': [0.0] * len_bydt, 'occupancy': [0.0] * len_bydt}
bydt_data_df = DataFrame(bydt_data, columns=['datetime', 'arrivals', 'departures', 'occupancy'])

for cat_df in all_cat_df:
    bydt_df_cat = pd.concat([cat_df, bydt_data_df],axis=1)
    bydt_df = pd.concat([bydt_df, bydt_df_cat])

In [32]:
bydt_df.head()

,PatType,Severity,datetime,arrivals,departures,occupancy
0,IVT,1,1996-01-01 00:00:00,0,0,0
1,IVT,1,1996-01-01 01:00:00,0,0,0
2,IVT,1,1996-01-01 02:00:00,0,0,0
3,IVT,1,1996-01-01 03:00:00,0,0,0
4,IVT,1,1996-01-01 04:00:00,0,0,0


In [33]:
bydt_df.tail()

,PatType,Severity,datetime,arrivals,departures,occupancy
2155,OTH,2,1996-03-30 19:00:00,0,0,0
2156,OTH,2,1996-03-30 20:00:00,0,0,0
2157,OTH,2,1996-03-30 21:00:00,0,0,0
2158,OTH,2,1996-03-30 22:00:00,0,0,0
2159,OTH,2,1996-03-30 23:00:00,0,0,0


In [34]:
catfield

['PatType', 'Severity']

In [35]:
midx_fields = catfield.copy()

In [36]:
midx_fields.append('datetime')

In [37]:
midx_fields

['PatType', 'Severity', 'datetime']

In [39]:
bydt_df['day_of_week'] = bydt_df['datetime'].map(lambda x: x.weekday())
bydt_df['bin_of_day'] = bydt_df['datetime'].map(lambda x: hmlib.bin_of_day(x, bin_size_minutes))
bydt_df['bin_of_week'] = bydt_df['datetime'].map(lambda x: hmlib.bin_of_week(x, bin_size_minutes))


In [40]:
bydt_df.head()

,PatType,Severity,datetime,arrivals,departures,occupancy,day_of_week,bin_of_day,bin_of_week
0,IVT,1,1996-01-01 00:00:00,0,0,0,0,0,0
1,IVT,1,1996-01-01 01:00:00,0,0,0,0,1,1
2,IVT,1,1996-01-01 02:00:00,0,0,0,0,2,2
3,IVT,1,1996-01-01 03:00:00,0,0,0,0,3,3
4,IVT,1,1996-01-01 04:00:00,0,0,0,0,4,4


In [41]:
bydt_df.tail()

,PatType,Severity,datetime,arrivals,departures,occupancy,day_of_week,bin_of_day,bin_of_week
2155,OTH,2,1996-03-30 19:00:00,0,0,0,5,19,139
2156,OTH,2,1996-03-30 20:00:00,0,0,0,5,20,140
2157,OTH,2,1996-03-30 21:00:00,0,0,0,5,21,141
2158,OTH,2,1996-03-30 22:00:00,0,0,0,5,22,142
2159,OTH,2,1996-03-30 23:00:00,0,0,0,5,23,143


In [42]:
midx_fields = catfield.copy()
midx_fields.append('datetime')
bydt_df.set_index(midx_fields, inplace=True, drop=False)

bydt_df.sortlevel(inplace=True)

In [43]:
bydt_df.head()

PatType  Severity            datetime  \
PatType Severity datetime                                                    
ART     1        1996-01-01 00:00:00     ART         1 1996-01-01 00:00:00   
                 1996-01-01 01:00:00     ART         1 1996-01-01 01:00:00   
                 1996-01-01 02:00:00     ART         1 1996-01-01 02:00:00   
                 1996-01-01 03:00:00     ART         1 1996-01-01 03:00:00   
                 1996-01-01 04:00:00     ART         1 1996-01-01 04:00:00   

                                      arrivals  departures  occupancy  \
PatType Severity datetime                                               
ART     1        1996-01-01 00:00:00         0           0          0   
                 1996-01-01 01:00:00         0           0          0   
                 1996-01-01 02:00:00         0           0          0   
                 1996-01-01 03:00:00         0           0          0   
                 1996-01-01 04:00:00         0           0          0   

                                      day_of_week  bin_of_day  bin_of_week  
PatType Severity datetime                                                   
ART     1        1996-01-01 00:00:00            0           0            0  
                 1996-01-01 01:00:00            0           1            1  
                 1996-01-01 02:00:00            0           2            2  
                 1996-01-01 03:00:00            0           3            3  
                 1996-01-01 04:00:00            0           4            4

In [44]:
bydt_df.index.names

FrozenList(['PatType', 'Severity', 'datetime'])

In [23]:
row = stops_df.iloc[0]

In [52]:
row

PatID                          1
InRoomTS     1996-01-01 07:44:00
OutRoomTS    1996-01-01 08:50:00
PatType                      IVT
Severity                       1
Name: 0, dtype: object

In [54]:
row['PatID']

1

In [55]:
catfield

['PatType', 'Severity']

In [56]:
row[catfield]

PatType     IVT
Severity      1
Name: 0, dtype: object

In [57]:
type(catfield)

list

In [58]:
type(row[catfield])

pandas.core.series.Series

In [59]:
tuple(row[catfield])

('IVT', 1)

In [60]:
cat = tuple(row[catfield])

In [61]:
cat

('IVT', 1)

In [62]:
str(cat)

"('IVT', 1)"

In [70]:
infield = in_fld_name
outfield = out_fld_name

intime_raw = row[infield]
outtime_raw = row[outfield]

cat = tuple(row[catfield])

intime = hmlib.to_the_second(intime_raw)
outtime = hmlib.to_the_second(outtime_raw)

indtbin = hmlib.dt_floor(intime, bin_size_minutes)
outdtbin = hmlib.dt_floor(outtime, bin_size_minutes)

inout_occ_frac = hmlib.occ_frac([intime, outtime], bin_size_minutes, 1)

numbins = hmlib.numbins(indtbin, outdtbin, bin_size_minutes)
dtbin = indtbin

bydt_df.at[(*cat, indtbin), 'occupancy'] += inout_occ_frac[0]
bydt_df.at[(*cat, indtbin), 'arrivals'] += 1.0
bydt_df.at[(*cat, outdtbin), 'departures'] += 1.0

In [71]:
inout_occ_frac[0]

0.2666666666666667

In [72]:
(*cat, indtbin)

('IVT', 1, Timestamp('1996-01-01 07:00:00'))

In [73]:
bydt_df.at[(*cat, indtbin), 'occupancy']

0.26666666666666672

In [21]:


scenario = 'iss17_test1'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
catfield = ['PatType','Severity']
start_analysis_dt = '1/1/1996'
end_analysis_dt = '3/30/1996 23:45'

# Optional inputs
tot_fld_name = 'SSU'
bin_size_minutes = 60

In [37]:
bydt_test_df = bydatetime.make_bydatetime(stops_df, in_fld_name, out_fld_name, 
                                          start_analysis_dt, end_analysis_dt,
                                          catfield=catfield)

IVT
1
IVT
2
MYE
1
MYE
2
CAT
1
CAT
2
ART
1
ART
2
OTH
1
OTH
2


TypeError: string indices must be integers

In [25]:
mini_df = stops_df[0:10]

In [1]:
len(mini_df)

NameError: name 'mini_df' is not defined

In [ ]:
for row in mini_df.iterrows():
    print(row)
    print(row[0],row[1])
    print(row[1]['PatType'])

In [11]:
catfield

['PatType', 'Severity']

In [12]:
print('_'.join(catfield))

PatType_Severity
